# Calculate Average Potential Carbon Sequestration in Ecozones, Countries, and Continents

This script calculates statistics on potential carbon sequestration rates over ecozones, countries, and continents. We use per-pixel estimates of the average potential carbon sequestration rate in Mg C/ha/year and per-pixel variance of model estimates from [Cook-Patton et. al 2020](https://www.nature.com/articles/s41586-020-2686-x) to calculate the regional average potential carbon sequestration rate, standard deviation of model variance, and the 95% cofidence interval for the average rate in the region.

Ecozones sourced from FAO [Global Ecological Zones (GEZ) mapping](https://www.fao.org/forest-resources-assessment/remote-sensing/global-ecological-zones-gez-mapping/en/) which can be [downlaoded here](https://data.apps.fao.org/map/catalog/static/api/records/2fb209d0-fd34-4e5e-a3d8-a13c241eb61b).

Country boundaries are sourced from [GADM version 3.6](https://gadm.org/download_world36.html)

Ecozones and country boundaries were intersected using QGIS.


In [2]:
import sys
import ee
import numpy as np
import pandas as pd
import time

In [3]:
#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [8]:
# load in image collection from Data Lab repository
imageCollection = ee.ImageCollection('projects/wri-datalab/GFWCarbonSequestrationYoungForests')

# save projection information
projection = imageCollection.first().projection().getInfo()
projection_gee = imageCollection.first().projection()
crs = projection.get('crs')
crs_transform = projection.get('transform')

# convert image collection to image
image = imageCollection.toBands()
# select bands and rename them
image = image.select(['AboveGround_CoefVariation_b1', 'AboveGround_Lower_CI_b1', 'AboveGround_Mean_b1', 
                      'AboveGround_StandardDeviation_b1', 'AboveGround_Upper_CI_b1', 'AboveGround_Variance_b1', 
                      'BelowGround_Mean_b1', 'BelowGround_Variance_b1'])
image = image.rename(['AboveGround_CoefVariation', 'AboveGround_Lower_CI', 'AboveGround_Mean', 
                      'AboveGround_StandardDeviation', 'AboveGround_Upper_CI', 'AboveGround_Variance', 
                      'BelowGround_Mean', 'BelowGround_Variance'])

def calculate_rate_statistics(feature_collection, region_name, file_name, folder_name):
    """
    Function to calculate statistics on potential carbon sequestion rate over a given feature collection,
    including the mean, minimum, and maximum of rates in each region, the 95% confidence interval for the 
    mean rate in each region, and the standard deviation of the rate in the region. 

    Parameters:
    feature_collection (ee.FeatureCollection): feature collection over which to calculate statistics
    region_name (String): name to be included in column names (e.g., 'Average Rate in X' where X is region_name)
    file_name (String): file name of CSV to export to in Google Drive
    folder_name (String): folder name to export to in Google Drive

    Returns:
    ee.FeatureCollection of regions with calculated potential carbon sequestration rates
    """
    # create combined reducer for calculating the mean, minimum, and maximum of rates in region
    min_max_reducers = ee.Reducer.mean().unweighted().combine(ee.Reducer.min().unweighted(), sharedInputs=True)
    min_max_reducers = min_max_reducers.combine(ee.Reducer.max().unweighted(), sharedInputs=True)
    
    # create combined reducer for calculating sum of variance and count of pixels, which will be used
    # to calculate confidence intervals for the average rate in the region
    sum_count_reducers = ee.Reducer.sum().unweighted().combine(ee.Reducer.count().unweighted(), sharedInputs=True)

    # calculate mean, minimum, and maximum of rates over regions
    reduce_region_results = image.select(['AboveGround_Mean']).reduceRegions(
                                    feature_collection, 
                                    reducer=min_max_reducers.setOutputs(['AboveGround_Mean_mean','AboveGround_Mean_min','AboveGround_Mean_max']), 
                                    crs=crs, crsTransform=crs_transform, 
                                    tileScale=16)
    # calculate sum of model variance and count of pixels over regions
    reduce_region_results = image.select(['AboveGround_Variance']).reduceRegions(
                                    reduce_region_results, 
                                    reducer=sum_count_reducers.setOutputs(['AboveGround_Variance_sum','AboveGround_Variance_count']), 
                                    crs=crs, crsTransform=crs_transform, 
                                    tileScale=16)
    
    # function to format results from reduce regions and calculate confidence intervals from
    # regional variance sum and pixel count
    def format_results(feature):
        feature = ee.Feature(feature)
        # select statistics
        mean = ee.Number(feature.get('AboveGround_Mean_mean'))
        minimum = ee.Number(feature.get('AboveGround_Mean_min'))
        maximum = ee.Number(feature.get('AboveGround_Mean_max'))
        variance_sum = ee.Number(feature.get('AboveGround_Variance_sum'))
        variance_count = ee.Number(feature.get('AboveGround_Variance_count'))
        
        # calculate standard deviation and 95% confidence intervals
        # with GEE, Null values can cause errors in calculations, so use If statement to catch
        # Null values and otherwise return value
        
        # calculate variance
        variance = ee.Algorithms.If(variance_sum, variance_sum.divide(variance_count.pow(2)), None)
        
        # calculate standard deviation taking the square root of the variance
        standard_deviation = ee.Algorithms.If(variance_sum, ee.Number(variance).sqrt(), None)
        
        # calculate 95% confidence itnervals
        lower_bound = ee.Algorithms.If(mean, mean.subtract(ee.Number(standard_deviation).multiply(1.96)), None)
        upper_bound = ee.Algorithms.If(mean, mean.add(ee.Number(standard_deviation).multiply(1.96)), None)
        
        # save to features with formatted names
        feature = feature.set({
            'Above Ground Rate Average in '+region_name: mean,
            'Above Ground Rate Minimum in '+region_name: minimum,
            'Above Ground Rate Maximum in '+region_name: maximum,
            'AboveGroundSequestration_Unit'+region_name: 'Mg C/ha/year',
            'Variance of Above Ground Rate Average in '+region_name:variance,
            'Standard Deviation of Above Ground Rate Average in '+region_name:standard_deviation,
            'Lower Bound of 95% Confidence Interval of Above Ground Rate Average in '+region_name:lower_bound,
            'Upper Bound of 95% Confidence Interval of Above Ground Rate Average in '+region_name:upper_bound,
            'Number of Pixels in ': variance_count
        })
        
        # remove property names from reduce regions calculation
        property_names = feature.propertyNames()
        property_names = property_names.removeAll(ee.List(['AboveGround_Mean_mean',
                                                'AboveGround_Mean_min',
                                                'AboveGround_Mean_max',
                                                'AboveGround_Variance_sum',
                                                'AboveGround_Variance_count']))
        # drop geometry so file can be easily read in Excel
        feature = feature.select(property_names,retainGeometry=False)
        return feature
    
    # loop over results to format and return
    results = reduce_region_results.map(format_results)
    results = ee.FeatureCollection(results)
    # export to Google Drive
    export_results_task = ee.batch.Export.table.toDrive(
        collection = results, 
        folder = folder_name,
        description = file_name, 
        fileNamePrefix = file_name)

    export_results_task.start()
    


In [10]:
# define regions asset
regions = ee.FeatureCollection('projects/wri-datalab/GFWCarbonSequestrationYoungForestsBoundaries/FAO_GEZ_Ecozones_By_Country')
# define name for columns
region_name = 'Ecozone and Country'
# define file name to export to
output_name = 'CarbonSequestration_AboveGroundAverage_Ecozone_Country_GEE'
# define folder to export to
folder_name = 'CarbonRateCalculationsForIPCC'

# rename columns from FAO Ecozones and GADM country boundaries to be easily readable
regions = ee.FeatureCollection(regions.map(lambda x: ee.Feature(x).select(['GID_0','NAME_0','gez_abbrev','gez_code','gez_name','RootShoot'], 
       ['ISO Code','Country','Ecozone Abbreviation','Ecozone Code','Ecozone Name','RootShoot'], True)))

# calculate rates for regions
calculate_rate_statistics(regions,region_name,output_name,folder_name)


In [36]:
# define regions asset
regions = ee.FeatureCollection('projects/wri-datalab/GFWCarbonSequestrationYoungForestsBoundaries/FAO_GEZ_Ecozones_By_Continent')
# define name for columns
region_name = 'Ecozone and Continent'
# define file name to export to
output_name = 'CarbonSequestration_AboveGroundAverage_Ecozone_Continent_GEE'
# define folder to export to
folder_name = 'CarbonRateCalculationsForIPCC'

#rename columns from FAO Ecozones and continent boundaries to be easily readable
regions = ee.FeatureCollection(regions.map(lambda x: ee.Feature(x).select(['REGION','gez_abbrev','gez_code','gez_name','RootShoot'], 
      ['Continent','Ecozone Abbreviation','Ecozone Code','Ecozone Name','RootShoot'], True)))

#calculate rates for regions
calculate_rate_statistics(regions,region_name,output_name,folder_name)

In [ ]:
# define regions asset
regions = ee.FeatureCollection('projects/wri-datalab/GFWCarbonSequestrationYoungForestsBoundaries/FAO_GEZ_Ecozones')
# define name for columns
region_name = 'Ecozone'
# define file name to export to
output_name = 'CarbonSequestration_AboveGroundAverage_Ecozone_GEE'
# define folder to export to
folder_name = 'CarbonRateCalculationsForIPCC'

#rename columns from FAO Ecozones and continent boundaries to be easily readable
regions = ee.FeatureCollection(regions.map(lambda x: ee.Feature(x).select(['gez_abbrev','gez_code','gez_name','RootShoot'], 
      ['Ecozone Abbreviation','Ecozone Code','Ecozone Name','RootShoot'], True)))

#calculate rates for regions
calculate_rate_statistics(regions,region_name,output_name,folder_name)